In [1]:
user = {"username": "jose", "access_level": "guest"}

In [19]:
def get_admin_password():
    return "1234"

**That's risky largely due to false access level authentication**

In [3]:
get_admin_password()

'1234'

### Try alternative secure_get_admin() function to be more secure

In [4]:
def secure_get_admin():
    if user['access_level'] == 'admin':
        print(get_admin_password())

**secure_get_admin() function is more secure**

In [5]:
secure_get_admin()

**but get_admin_password() is still around, and anyone can still call it directly.**

In [6]:
get_admin_password()

'1234'

### Try to get rid of get_admin_password() to be more secure

In [10]:
def secure_function(func):
    if user['access_level'] == 'admin':
        return func

In [12]:
user = {"username": "bob", "access_level": "admin"}

get_admin_password = secure_function(get_admin_password)
get_admin_password()

'1234'

**When we run `secure_function`, we have to check the user's access level first, because if the user was not an admin, the function will not `return func`. Therefore `get_admin_password` is set to `None`.**

In [14]:
user = {"username": "jose", "access_level": "guest"}

get_admin_password = secure_function(get_admin_password)
get_admin_password()

TypeError: 'NoneType' object is not callable

### Simple decorator to avoid errors

In [21]:
def make_secure(func):
    def secure_function():
        if user['access_level'] == 'admin':
            return func()
        else:
            return f'No admin permissions for {user["username"]}'
        
    return secure_function

In [17]:
user = {"username": "jose", "access_level": "guest"}

get_admin_password = make_secure(get_admin_password)
get_admin_password()

In [20]:
user = {"username": "jose", "access_level": "admin"}

get_admin_password = make_secure(get_admin_password)
get_admin_password()

'1234'

# The @ syntax for decorators

In [22]:
def make_secure(func):
    def secure_function():
        if user['access_level'] == 'admin':
            return func()
        else:
            return f'No admin permissions for {user["username"]}'
        
    return secure_function


@make_secure
def get_admin_password():
    return "1234"

In [23]:
user = {"username": "jose", "access_level": "guest"}

get_admin_password()

'No admin permissions for jose'

In [24]:
user = {"username": "jose", "access_level": "admin"}

get_admin_password()

'1234'

**The information of get_admin_password() will be lost**

In [25]:
get_admin_password.__name__

'secure_function'

### Keep the documentation of get_admin_password()

In [26]:
import functools

In [27]:
def make_secure(func):
    @functools.wraps(func)
    def secure_function():
        if user['access_level'] == 'admin':
            return func()
        else:
            return f'No admin permissions for {user["username"]}'
        
    return secure_function


@make_secure
def get_admin_password():
    return "1234"

In [28]:
get_admin_password.__name__

'get_admin_password'

### Decorating functions with parameters

**Can't work because the inner secure_function() doesn't accept any parameters, so the panel argument in get_password() can't be passed to secure_function()**

In [32]:
def make_secure(func):
    @functools.wraps(func)
    def secure_function():
        if user['access_level'] == 'admin':
            return func()
        else:
            return f'No admin permissions for {user["username"]}'
        
    return secure_function


@make_secure
def get_password(panel: str):
    if panel == 'admin':
        return '1234'
    elif panel == 'billing':
        return '****'
    
get_password('billing')

TypeError: secure_function() takes 0 positional arguments but 1 was given

**If secure_function() add parameter 'panel', the argument in get_password() can be sent to secure_function(), but**

**make_secure() decorator function will be no longer generic**

In [33]:
def make_secure(func):
    @functools.wraps(func)
    def secure_function(panel):
        if user['access_level'] == 'admin':
            return func(panel)
        else:
            return f'No admin permissions for {user["username"]}'
        
    return secure_function


@make_secure
def get_password(panel: str):
    if panel == 'admin':
        return '1234'
    elif panel == 'billing':
        return '****'
    
get_password('billing')

'****'

**Use \*args, \*\*kwargs arguments instead, in order for more generic make_secure() decorator function**

In [31]:
def make_secure(func):
    @functools.wraps(func)
    def secure_function(*args, **kwargs):
        if user['access_level'] == 'admin':
            return func(*args, **kwargs)
        else:
            return f'No admin permissions for {user["username"]}'
        
    return secure_function


@make_secure
def get_password(panel: str):
    if panel == 'admin':
        return '1234'
    elif panel == 'billing':
        return '****'


get_password('billing')

'****'

### Decorators with parameters

**In the following strucute, decorator functions can't accept arguments to run if-else statements**

In [34]:
user = {"username": "jose", "access_level": "guest"}

def make_secure(func):
    @functools.wraps(func)
    def secure_function(*args, **kwargs):
        if user['access_level'] == 'admin':
            return func(*args, **kwargs)
        else:
            return f'No admin permissions for {user["username"]}'
        
    return secure_function

@make_secure
def get_admin_password():
    return 'admin: 1234'

@make_secure
def get_dashboard_paasword():
    return 'user: user_password'

**allow decorator function to be able to choose a proper function to return correct responses**

In [44]:
def make_secure(access_level):
    def decorator(func):
        @functools.wraps(func)
        def secure_function(*args, **kwargs):
            if user['access_level'] == access_level:
                return func(*args, **kwargs)
            else:
                return f'No {access_level} permissions for {user["username"]}'
        
        return secure_function
    return decorator


@make_secure('admin')
def get_admin_password():
    return 'admin: 1234'


@make_secure('guest')
def get_dashboard_paasword():
    return 'user: ****'


In [45]:
user = {"username": "jose", "access_level": "guest"}

print(get_admin_password())
print(get_dashboard_paasword())

No admin permissions for jose
user: ****


In [46]:
user = {"username": "jose", "access_level": "admin"}

print(get_admin_password())
print(get_dashboard_paasword())

admin: 1234
No guest permissions for jose


In [47]:
@make_secure('user')
def get_dashboard_paasword():
    return 'user: ****'

user = {"username": "jose", "access_level": "guest"}

print(get_admin_password())
print(get_dashboard_paasword())

No admin permissions for jose
No user permissions for jose
